In [40]:
import requests  # go out and fetch
import pandas as pd
import numpy as np
import time
from datetime import datetime
import re
import seaborn as sns


#function to iterate through files and have filename in the row for pandas
def nextfile(x):
    return(x)



def inlist(sentence):
    print(any(ele in sentence for ele in peeps))
    # res = [all([k in s for k in words]) for s in sentence]
    # return [sentence[i] for i in range(0, len(res)) if res[i]]

In [41]:
therapist = ["therapist:", "th:"]
patient = ["client:", "cl:"]

col_names = ['fname','therapist','sentence', 'timestamp'] 
df = pd.DataFrame(columns=col_names)
ts_pattern = re.compile('\[\d\d:\d\d\:\d\d\]')
df_append = pd.DataFrame(columns=col_names)



In [42]:

# fname = "S___Session_February_17__2014"
# fpath = '/Users/anjalee/Desktop/DemoFolder/pythonSubDir/session_transcripts/Client__S___Session_February_17__2014.txt'

fname = "A___Session_April_03__2014"
fpath = '/Users/anjalee/Desktop/DemoFolder/pythonSubDir/session_transcripts/Client__A___Session_April_03__2014.txt'


with open(fpath,'r') as f:
    for line in f:
        line = f.readline() # read line
        
        line = line.lower() # lowercase
        line = line.encode("ascii", "ignore") # rid of non-ascii, i know this can mush two words together if there's a '-' in the middle
        line = line.decode()
        
        if ts_pattern.search(line): # catch & remove timestamps
            ts = ts_pattern.findall(line)      
        else: ts = "no time"
        # re.sub(ts_pattern,'', line) #DOESNT WORK  extract all timestamps
        
        df_append = pd.DataFrame(columns=col_names) # clear df_append from last time
        if ('therapist:' in line) or ('th:' in line):
            df_append = pd.DataFrame([[fname,'1',line,ts]], columns=col_names) 

        elif ('client:' in line) or ('cl:' in line):           
            df_append = pd.DataFrame([[fname,'0',line,ts]], columns=col_names)           
        
        df = pd.concat([df, df_append]) #because append() is gone
        
        
        if not line:
            break
       

In [43]:
temp = "CLIENT: So (pause) it’s been very hard between Jennie [and I] (ph) the last few days. [0:01:04] I think—[we even talked about it] (ph) a little bit. I’ve been really, really angry about (pause)… I mean, it’s really functionally—the series of interactions that we’ve had about time hasn’t had too profound an impact on time that I have or don’t have. But it’s been very profound as a matter of legitimacy, [I mean] (ph) the legitimacy of working on this or that. [0:01:55] And this (pause) interaction we had on Monday, she sent me a really obnoxious e-mail while she was in the kitchen and I was in my office, asking me why I wasn’t working better and making more productive use of her parents, that I was being—I mean, there were a number of things that were upsetting. And I think I mentioned them on Tuesday. I think?"

In [44]:
tpat = 
re.sub(ts_pattern,'', temp)


SyntaxError: invalid syntax (593081010.py, line 1)